<a href="https://colab.research.google.com/github/dhanapalblr/chatbot/blob/master/berttest_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install tez
# !pip install torch==1.7.1
# !pip install transformers==3.5.1

In [ ]:
import pandas as pd
import tez
import torch
import torch.nn as nn
import transformers
from sklearn import metrics, model_selection, preprocessing
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.utils import shuffle
import torch.nn.functional as nnf

In [ ]:
# import os
# os.chdir('/content/drive/MyDrive/Work/Dan/Server_Run')

In [ ]:
class BERTBaseUncased(tez.Model):
    def __init__(self, num_train_steps, num_classes):
        super().__init__()
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-large-uncased", do_lower_case=True
        )
        self.bert = transformers.BertModel.from_pretrained("bert-large-uncased")
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(1024, num_classes)
        self.num_train_steps = num_train_steps
        self.step_scheduler_after = "batch"
    def fetch_optimizer(self):
        param_optimizer = list(self.named_parameters())
        no_decay = ["bias", "LayerNorm.bias"]
        optimizer_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.001,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]
        opt = AdamW(optimizer_parameters, lr=3e-5)
        return opt
    def fetch_scheduler(self):
        sch = get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps=0, num_training_steps=self.num_train_steps
        )
        return sch
    def loss(self, outputs, targets):
        if targets is None:
            return None
        return nn.CrossEntropyLoss()(outputs, targets)
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}
    def forward(self, ids, mask, token_type_ids, targets=None):
        _, o_2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        b_o = self.bert_drop(o_2)
        output = self.out(b_o)
        loss = self.loss(output, targets)
        acc = self.monitor_metrics(output, targets)
        return output, loss, acc

In [ ]:
import transformers
DEVICE = "cuda"
MAX_LEN = 64
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10
BERT_PATH = "bert-large-uncased"
MODEL_PATH = "Hiri_Model_Fold0.bin"
TRAINING_FILE = "lcstrain.csv"
TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)
d=torch.load("lmap1.bin")

In [ ]:
import time
import torch.nn as nn
import pandas as pd
MODEL = BERTBaseUncased(10000,len(d))

MODEL.load_state_dict(torch.load("Hiri_Model_Fold0.bin"),strict=False)
MODEL.to(DEVICE)
MODEL.eval()

BERTBaseUncased(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [ ]:
# In[23]:
# DEVICE = config.DEVICE
# PREDICTION_DICT = dict()
tokenizer = TOKENIZER
max_len = MAX_LEN
sent_csv=pd.read_excel("lcstest.xlsx").rename(columns={'intent':'intent1'})
sentence=sent_csv.utterance.values

maxindx,predictions,maxindx1,maxindx2,maxindx3,maxindx4,predictions1,predictions2,predictions3,predictions4=[],[],[],[],[],[],[],[],[],[]
top_p1,top_p2,top_p3,top_class1,top_class2,top_class3=[],[],[],[],[],[]

In [ ]:
for index,value in enumerate(sentence):
    print(value)
    review = str(value)
    review = " ".join(review.split())
    inputs = tokenizer.encode_plus(
            review, None, add_special_tokens=True, max_length=max_len
        )
    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]
    padding_length = max_len - len(ids)
    ids = ids + ([0] * padding_length)
    mask = mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)
    ids = torch.tensor(ids, dtype=torch.long).unsqueeze(0)
    mask = torch.tensor(mask, dtype=torch.long).unsqueeze(0)
    token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0)
    ids = ids.to(DEVICE, dtype=torch.long)
    token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
    mask = mask.to(DEVICE, dtype=torch.long)
    outputs = MODEL(ids=ids, mask=mask, token_type_ids=token_type_ids)
    maxidx=torch.argmax(outputs[0],keepdim=False)
    maxindx.append(int(maxidx))
    print(int(maxidx))
    
    print(d[int(maxidx)])  
    predictions.append(d[int(maxidx)])
    # outputs = torch.sigmoid(outputs).cpu().detach().numpy()
    print(str(outputs[0]))
    
    prob = nnf.softmax(outputs[0], dim=1)
    top_p, top_class = prob.topk(3, dim = 1)
    print("confidence score",top_p)
    print("confidence score1",top_p[0][0])
    
    top_p1.append(float(top_p[0][0]))
    top_p2.append(float(top_p[0][1]))
    top_p3.append(float(top_p[0][2]))
    print("top_class",top_class)
    print("top_class1",top_class[0][0])
    
    top_class1.append(int(top_class[0][0]))
    top_class2.append(int(top_class[0][1]))
    top_class3.append(int(top_class[0][2]))
    
    predictions1.append(d[int(top_class[0][0])])
    predictions2.append(d[int(top_class[0][1])])
    predictions3.append(d[int(top_class[0][2])])
        
sent_csv["pred_idx"]=pd.DataFrame(maxindx)
sent_csv["pred_values"]=pd.DataFrame(predictions)
sent_csv["top_p1"]=pd.DataFrame(top_p1)
sent_csv["top_p2"]=pd.DataFrame(top_p2)
sent_csv["top_p3"]=pd.DataFrame(top_p3)
sent_csv["top_class1"]=pd.DataFrame(top_class1)
sent_csv["top_class2"]=pd.DataFrame(top_class2)
sent_csv["top_class3"]=pd.DataFrame(top_class3)
sent_csv["predictions1"]=pd.DataFrame(predictions1)
sent_csv["predictions2"]=pd.DataFrame(predictions2)
sent_csv["predictions3"]=pd.DataFrame(predictions3)

sent_csv.to_csv("Hiri_Model_Fold_0.csv")

# In[ ]:

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Streaming output truncated to the last 5000 lines.
confidence score1 tensor(0.0196, device='cuda:0', grad_fn=<SelectBackward>)
top_class tensor([[127,  24,  22]], device='cuda:0')
top_class1 tensor(127, device='cuda:0')
What must I know about UK Privacy Laws?
127
sanctions.lcs_sanctions_sanctions_contact_escalation
tensor([[ 1.0527, -0.9824, -0.0481, -0.6963,  0.7698, -0.3613,  0.9448, -0.3140,
          0.7757, -0.4341, -0.6873, -0.5155,  0.5796, -0.5128, -0.1932,  0.2448,
          0.2691, -0.3548,  0.0424, -0.2215, -0.5075, -1.0597,  1.1249,  0.5919,
          1.1461,  0.0723,  0.6962, -0.4727, -0.0215,  0.1035,  0.7044, -0.2352,
         -0.2803,  0.4621, -0.1976, -0.3176, -0.0549, -0.1908, -0.7433, -0.5251,
         -0.3070, -0.0104, -0.6499, -0.2259, -0.1656,  0.9539, -0.1427, -0.1197,
         -0.3555, -0.1464, -0.2831,  0.1009, -0.1264, -0.4715, -0.2031, -0.5449,
         -0.6233, -0.0303, -0.1495,  0.6808,  0.5118,  0.5409, -0.1552, -0.2069,
         -0.4519,  0.8016, -0.2707,